In [7]:
from models import VCLModel, VanillaMLP
from models.coreset import RandomCoreset, UncertaintyCoreset
from datasets import PermutedMnist

import numpy as np

import os

import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
NUM_EPOCHS = 50
LR = 1e-3
BATCH_SIZE = 256
TRAIN_NUM_SAMPLES = 10
TEST_NUM_SAMPLES = 100
UNCERTAINTY_NUM_SAMPLES = 100

INIT_MLP = False

MLP_INIT_DESC = 'withoutmlpinit' if not INIT_MLP else 'withmlpinit'

random_seed = 1
torch.manual_seed(random_seed + 1)
np.random.seed(random_seed)

model_base_name = 'only_asdf_bad_uncertainty_epochs_vcl_lr_{}_{}_batch_{}_uncertainty_coreset_method_{}_uncertainty_coreset_size_{}_coresetsize_{}_epochs_{}_run_{}_no_tasks_{}'

In [9]:
no_tasks = 10

dataset = PermutedMnist(no_tasks)

In [10]:
train_loaders, test_loaders = dataset.get_tasks(batch_size=BATCH_SIZE)
no_tasks = dataset.no_tasks

# Modify as need be.
NO_RUNS = 3
epochs = [50]
methods = ['prob_std', 'entropy']
coreset_pairs = [(1000, 4000), (250, 4750)]

for method in methods:
    UNCERTAINTY_CORESET_METHOD = method
    for epoch in epochs:
        NUM_EPOCHS = epoch
        for (uncertainty_coreset_size, coreset_size) in coreset_pairs:
            UNCERTAINTY_CORESET_SIZE = uncertainty_coreset_size
            CORESET_SIZE = coreset_size
            for run_number in range(NO_RUNS):
                model_name = model_base_name.format(LR, MLP_INIT_DESC, BATCH_SIZE, UNCERTAINTY_CORESET_METHOD, UNCERTAINTY_CORESET_SIZE, CORESET_SIZE, NUM_EPOCHS, run_number, no_tasks) 
               
                LOG_DIR = 'logs/'
                os.makedirs(LOG_DIR, exist_ok=True)
                
                LOG_FILE_NAME = os.path.join(LOG_DIR, model_name) + '.txt'
                log_file_handler = open(LOG_FILE_NAME, "w")
                print("Run Number: {}. Log File: {}".format(run_number, LOG_FILE_NAME))
                
                uncertainty_coreset_selector = UncertaintyCoreset(dataset, coreset_size=CORESET_SIZE)
                train_loaders = uncertainty_coreset_selector.get_train_loaders(batch_size=BATCH_SIZE)
                
                model = VCLModel(784, 10, [100, 100])
                accuracies = np.zeros((no_tasks, no_tasks))
                
                for i in range(no_tasks):
                    train_loader = train_loaders[i]
                    
                    if INIT_MLP and i == 0:
                        print("Training MLP model to init first task")
                        mlp = VanillaMLP(784, 10, [100, 100])
                        mlp.train_model(NUM_EPOCHS, train_loader, LR)
                        model.init_mle(mlp)
                    
                    model.train_model(NUM_EPOCHS, train_loader, LR, TRAIN_NUM_SAMPLES)
                    
                    model.update_priors()
                    
                    # Train on coreset after training on non-coreset.
                    uncertainty_coreset_selector.update_uncertainty_coreset_and_get_loader(
                        task_idx=i,
                        batch_size=BATCH_SIZE,
                        training_task_loader=train_loader,
                        model=model,
                        uncertainty_coreset_size=UNCERTAINTY_CORESET_SIZE,
                        no_samples=UNCERTAINTY_NUM_SAMPLES,
                        type=UNCERTAINTY_CORESET_METHOD,
                    )
                    
                    uncertainty_loader = uncertainty_coreset_selector.get_uncertainty_loader(BATCH_SIZE)
                    coreset_loader = uncertainty_coreset_selector.get_main_coreset_loader(i, BATCH_SIZE)
                    
                    model.train_model(20, uncertainty_loader, LR, TRAIN_NUM_SAMPLES)
                    model.train_model(50, coreset_loader, LR, TRAIN_NUM_SAMPLES)
    
                    task_accs = []
                    for j in range(i + 1):
                        test_loader = test_loaders[j]
                        accuracy = model.get_accuracy(test_loader, 100)
                        
                        msg = "[Number Tasks: {}]: Task {} Accuracy: {}".format(j + 1, j + 1, accuracy ) 
                        log_file_handler.write(msg + '\n')
                        log_file_handler.flush()
                        print(msg)
                        
                        task_accs.append(accuracy)
                        accuracies[i][j] = accuracy
                    
                    msg = "Task {} / {}. Mean Accuracy: {}".format(i + 1, no_tasks, np.mean(task_accs))
                    log_file_handler.write(msg + '\n')
                    log_file_handler.flush()
                    print(msg)
                log_file_handler.close()

Run Number: 0. Log File: logs/only_asdf_bad_uncertainty_epochs_vcl_lr_0.001_withoutmlpinit_batch_256_uncertainty_coreset_method_prob_std_uncertainty_coreset_size_1000_coresetsize_4000_epochs_50_run_0_no_tasks_10.txt


Total Loss: 0.9816607871012056, KL: 0.8970230205418313, Lik Loss: 0.08463776519853775: 100%|██████████| 50/50 [05:24<00:00,  6.48s/it]
Total Loss: 0.8905361145734787, KL: 0.004636829835362732, Lik Loss: 0.8858992755413055: 100%|██████████| 20/20 [00:04<00:00,  4.38it/s] 
Total Loss: 0.05303544574417174, KL: 0.019002859946340322, Lik Loss: 0.0340325862634927: 100%|██████████| 50/50 [00:33<00:00,  1.48it/s]   


[Number Tasks: 1]: Task 1 Accuracy: 0.9807
Task 1 / 10. Mean Accuracy: 0.9807


Total Loss: 0.2922185176991981, KL: 0.12689006199303282, Lik Loss: 0.16532845652266726: 100%|██████████| 50/50 [06:07<00:00,  7.34s/it] 
Total Loss: 1.0594252049922943, KL: 0.02197649679146707, Lik Loss: 1.0374486967921257: 100%|██████████| 20/20 [00:05<00:00,  3.44it/s] 
Total Loss: 0.08818820072337985, KL: 0.029811659653205425, Lik Loss: 0.05837654089555144: 100%|██████████| 50/50 [01:07<00:00,  1.35s/it] 


[Number Tasks: 1]: Task 1 Accuracy: 0.9781
[Number Tasks: 2]: Task 2 Accuracy: 0.9665
Task 2 / 10. Mean Accuracy: 0.9722999999999999


...

[Number Tasks: 1]: Task 1 Accuracy: 0.9699
[Number Tasks: 2]: Task 2 Accuracy: 0.9584
[Number Tasks: 3]: Task 3 Accuracy: 0.9621
[Number Tasks: 4]: Task 4 Accuracy: 0.9641
[Number Tasks: 5]: Task 5 Accuracy: 0.9636
[Number Tasks: 6]: Task 6 Accuracy: 0.9611
[Number Tasks: 7]: Task 7 Accuracy: 0.9626
[Number Tasks: 8]: Task 8 Accuracy: 0.9627
Task 8 / 10. Mean Accuracy: 0.9630624999999999


Total Loss: 0.29778824036044105, KL: 0.1336816038798403, Lik Loss: 0.16410663789483132: 100%|██████████| 50/50 [03:52<00:00,  4.65s/it] 
Total Loss: 1.2115289105309381, KL: 0.0720037470261256, Lik Loss: 1.1395251618491278: 100%|██████████| 20/20 [00:05<00:00,  3.85it/s]  
Total Loss: 0.18612153765684117, KL: 0.04418032645376142, Lik Loss: 0.14194121135922963: 100%|██████████| 50/50 [03:12<00:00,  3.84s/it] 


[Number Tasks: 1]: Task 1 Accuracy: 0.9689
[Number Tasks: 2]: Task 2 Accuracy: 0.9546
[Number Tasks: 3]: Task 3 Accuracy: 0.9591
[Number Tasks: 4]: Task 4 Accuracy: 0.9582
[Number Tasks: 5]: Task 5 Accuracy: 0.9605
[Number Tasks: 6]: Task 6 Accuracy: 0.9585
[Number Tasks: 7]: Task 7 Accuracy: 0.9598
[Number Tasks: 8]: Task 8 Accuracy: 0.961
[Number Tasks: 9]: Task 9 Accuracy: 0.9624
Task 9 / 10. Mean Accuracy: 0.9603333333333334


Total Loss: 0.3080089978735756, KL: 0.13925664126873016, Lik Loss: 0.1687523570187666: 100%|██████████| 50/50 [04:07<00:00,  4.94s/it]  
Total Loss: 1.239158523082733, KL: 0.07705640867352485, Lik Loss: 1.1621021270751952: 100%|██████████| 20/20 [00:05<00:00,  3.53it/s] 
Total Loss: 0.19924460976354538, KL: 0.04606843617574502, Lik Loss: 0.1531761741285683: 100%|██████████| 50/50 [03:23<00:00,  4.06s/it]  


[Number Tasks: 1]: Task 1 Accuracy: 0.9644
[Number Tasks: 2]: Task 2 Accuracy: 0.9536
[Number Tasks: 3]: Task 3 Accuracy: 0.9561
[Number Tasks: 4]: Task 4 Accuracy: 0.9573
[Number Tasks: 5]: Task 5 Accuracy: 0.9567
[Number Tasks: 6]: Task 6 Accuracy: 0.9555
[Number Tasks: 7]: Task 7 Accuracy: 0.9583
[Number Tasks: 8]: Task 8 Accuracy: 0.9581
[Number Tasks: 9]: Task 9 Accuracy: 0.9589
[Number Tasks: 10]: Task 10 Accuracy: 0.9581
Task 10 / 10. Mean Accuracy: 0.9577
